In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

import tensorflow as tf
euclidean_dist_module = tf.load_op_library("euclidean_dist.so")
euclidean_dist = euclidean_dist_module.euclidean_dist
euclidean_dist_grad = euclidean_dist_module.euclidean_dist_grad

from tensorflow.python.framework import ops
@ops.RegisterGradient("EuclideanDist")
def _EuclideanDistGrad(op, grad):
    a = op.inputs[0]
    b = op.inputs[1]
    y = op.outputs[0]  # y = 0.5 * b / conj(a)
    xGrad, cGrad = euclidean_dist_grad(a,b,y,grad)
    return xGrad, cGrad

import time


def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def RBFEuclidean(x, C):
    """Computes distance from cluster centers defined in input C
    
    Both outdim and indim should be integers.
    """
    return -euclidean_dist(x,C)

sess = tf.InteractiveSession()

x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

W_fc2 = weight_variable([784, 10])
b_fc2 = bias_variable([10])

y_conv = RBFEuclidean(x, W_fc2) + b_fc2

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.initialize_all_variables())
start = time.time()
for i in range(20000):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1]})
    print("\rstep %d, training accuracy %g"%(i, train_accuracy), end="" if i%1000 else "\n")
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})

elapsed = time.time()-start
print("\nTraining took {}".format(elapsed))
print("\ntest accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
step 0, training accuracy 0.1
step 1000, training accuracy 0.8
step 2000, training accuracy 0.76
step 3000, training accuracy 0.88
step 4000, training accuracy 0.74
step 5000, training accuracy 0.82
step 6000, training accuracy 0.82
step 7000, training accuracy 0.88
step 8000, training accuracy 0.8
step 9000, training accuracy 0.8
step 10000, training accuracy 0.92
step 11000, training accuracy 0.66
step 12000, training accuracy 0.8
step 13000, training accuracy 0.88
step 14000, training accuracy 0.9
step 15000, training accuracy 0.84
step 16000, training accuracy 0.86
step 17000, training accuracy 0.8
step 18000, training accuracy 0.86
step 19000, training accuracy 0.9
step 19900, training accuracy 0.92
Training took 25.26233458518982

test accuracy 0.8681
